In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
!unzip "/content/drive/MyDrive/ViTskinmodel.zip"

Archive:  /content/drive/MyDrive/ViTskinmodel.zip
   creating: content/vit-base-SKINMODEL/
  inflating: content/vit-base-SKINMODEL/all_results.json  
  inflating: content/vit-base-SKINMODEL/preprocessor_config.json  
  inflating: content/vit-base-SKINMODEL/training_args.bin  
  inflating: content/vit-base-SKINMODEL/trainer_state.json  
  inflating: content/vit-base-SKINMODEL/config.json  
   creating: content/vit-base-SKINMODEL/checkpoint-12000/
  inflating: content/vit-base-SKINMODEL/checkpoint-12000/scheduler.pt  
  inflating: content/vit-base-SKINMODEL/checkpoint-12000/preprocessor_config.json  
  inflating: content/vit-base-SKINMODEL/checkpoint-12000/training_args.bin  
  inflating: content/vit-base-SKINMODEL/checkpoint-12000/rng_state.pth  
  inflating: content/vit-base-SKINMODEL/checkpoint-12000/trainer_state.json  
  inflating: content/vit-base-SKINMODEL/checkpoint-12000/config.json  
  inflating: content/vit-base-SKINMODEL/checkpoint-12000/optimizer.pt  
  inflating: content/vi

In [ ]:
!unzip "/content/drive/MyDrive/deitskinmodel.zip"

Archive:  /content/drive/MyDrive/deitskinmodel.zip
   creating: content/deit-base-SKINMODEL/
  inflating: content/deit-base-SKINMODEL/test_results.json  
  inflating: content/deit-base-SKINMODEL/all_results.json  
  inflating: content/deit-base-SKINMODEL/trainer_state.json  
  inflating: content/deit-base-SKINMODEL/preprocessor_config.json  
  inflating: content/deit-base-SKINMODEL/train_results.json  
  inflating: content/deit-base-SKINMODEL/pytorch_model.bin  
  inflating: content/deit-base-SKINMODEL/config.json  
  inflating: content/deit-base-SKINMODEL/training_args.bin  
   creating: content/deit-base-SKINMODEL/runs/
   creating: content/deit-base-SKINMODEL/runs/Mar28_16-38-33_0d70a2437661/
  inflating: content/deit-base-SKINMODEL/runs/Mar28_16-38-33_0d70a2437661/events.out.tfevents.1680031319.0d70a2437661.1332.2  
   creating: content/deit-base-SKINMODEL/runs/Mar28_16-38-33_0d70a2437661/1680021546.130095/
  inflating: content/deit-base-SKINMODEL/runs/Mar28_16-38-33_0d70a2437661/1

In [ ]:
!unzip "/content/drive/MyDrive/dataset.zip"

Archive:  /content/drive/MyDrive/dataset.zip
   creating: content/dataset.hf/
   creating: content/dataset.hf/validation/
  inflating: content/dataset.hf/validation/data-00000-of-00001.arrow  
  inflating: content/dataset.hf/validation/state.json  
  inflating: content/dataset.hf/validation/dataset_info.json  
   creating: content/dataset.hf/test/
  inflating: content/dataset.hf/test/data-00000-of-00001.arrow  
  inflating: content/dataset.hf/test/state.json  
  inflating: content/dataset.hf/test/dataset_info.json  
   creating: content/dataset.hf/train/
  inflating: content/dataset.hf/train/data-00000-of-00007.arrow  
  inflating: content/dataset.hf/train/data-00006-of-00007.arrow  
  inflating: content/dataset.hf/train/data-00001-of-00007.arrow  
  inflating: content/dataset.hf/train/data-00003-of-00007.arrow  
  inflating: content/dataset.hf/train/data-00002-of-00007.arrow  
  inflating: content/dataset.hf/train/state.json  
  inflating: content/dataset.hf/train/data-00005-of-00007.

In [ ]:
!pip install datasets
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 KB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 16.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 30.2 MB/s eta 0:00

In [ ]:
import numpy as np
import math
from tqdm import tqdm

In [ ]:
from datasets import load_from_disk

ds = load_from_disk("/content/content/dataset.hf")
test_ds = ds["test"]

In [ ]:
labels = ds['train'].features['label'].names

In [ ]:
print(labels)

['Acne and Rosacea Photos', 'Actinic cheilitis', 'Normal Skin', 'acanthosis nigricans', 'actinic keratosis', 'alopecia', 'angiokeratoma', 'atopic dermatitis', 'atypical melanocytic proliferation', 'basal cell carcinoma', 'biting insects', 'bowens disease', 'bullous disease', 'candida', 'candidiasis', 'chondrodermatitis nodularis', 'ctcl', 'cutaneceous larva migrans', 'dermatofibroma', 'distal subungual onychomycosis', 'drug eruptions', 'eczema', 'epidermal cyst', 'fixed drug eruption', 'folliculitis', 'granuloma annulare', 'hemangioma', 'herpes', 'impetigo', 'intertrigo', 'keloids', 'keratoacanthoma', 'lentigo', 'lichen planus', 'lichenoid keratosis', 'lupus', 'melanocytic nevi', 'melanoma', 'molluscum contagiosum', 'necrobiosis lipoidica', 'neurofibromatosis', 'nevus', 'other connective tissue diseases', 'other lichen related diseases', 'other light diseases', 'other nail related diseases', 'other psoraisis related diseases', 'perleche', 'pigmented benign keratosis', 'pityriasis and r

In [ ]:
from transformers import ViTFeatureExtractor
from transformers import ViTForImageClassification

vit_model = ViTForImageClassification.from_pretrained(
    "/content/content/vit-base-SKINMODEL",
    num_labels=len(labels),
    id2label={str(i): c for i, c in enumerate(labels)},
    label2id={c: str(i) for i, c in enumerate(labels)},
    ignore_mismatched_sizes=True
)

deit_model = ViTForImageClassification.from_pretrained(
    "/content/content/deit-base-SKINMODEL",
    num_labels=len(labels),
    id2label={str(i): c for i, c in enumerate(labels)},
    label2id={c: str(i) for i, c in enumerate(labels)},
    ignore_mismatched_sizes=True
)

vit_feature_extractor = ViTFeatureExtractor.from_pretrained("/content/content/vit-base-SKINMODEL")

deit_feature_extractor = ViTFeatureExtractor.from_pretrained("/content/content/deit-base-SKINMODEL")

/usr/local/lib/python3.9/dist-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


In [ ]:
vit_model.config.id2label

{'0': 'Acne and Rosacea Photos',
 '1': 'Actinic cheilitis',
 '2': 'Normal Skin',
 '3': 'acanthosis nigricans',
 '4': 'actinic keratosis',
 '5': 'alopecia',
 '6': 'angiokeratoma',
 '7': 'atopic dermatitis',
 '8': 'atypical melanocytic proliferation',
 '9': 'basal cell carcinoma',
 '10': 'biting insects',
 '11': 'bowens disease',
 '12': 'bullous disease',
 '13': 'candida',
 '14': 'candidiasis',
 '15': 'chondrodermatitis nodularis',
 '16': 'ctcl',
 '17': 'cutaneceous larva migrans',
 '18': 'dermatofibroma',
 '19': 'distal subungual onychomycosis',
 '20': 'drug eruptions',
 '21': 'eczema',
 '22': 'epidermal cyst',
 '23': 'fixed drug eruption',
 '24': 'folliculitis',
 '25': 'granuloma annulare',
 '26': 'hemangioma',
 '27': 'herpes',
 '28': 'impetigo',
 '29': 'intertrigo',
 '30': 'keloids',
 '31': 'keratoacanthoma',
 '32': 'lentigo',
 '33': 'lichen planus',
 '34': 'lichenoid keratosis',
 '35': 'lupus',
 '36': 'melanocytic nevi',
 '37': 'melanoma',
 '38': 'molluscum contagiosum',
 '39': 'necr

In [ ]:
def norm(a):
  return min(max(a, 0), 1)

def run_vit(PIL_Image):
  inputs = vit_feature_extractor(images=PIL_Image, return_tensors="pt")
  outputs = vit_model(**inputs)
  logits = outputs.logits

  top_5_class_idx = np.array(logits.argsort())[0][::-1][:5]

  top_5_class = [(vit_model.config.id2label[str(class_idx)],
                norm( logits[0][class_idx].item()/10 ) ) for class_idx in top_5_class_idx]

  return top_5_class

def run_deit(PIL_Image):
  inputs = deit_feature_extractor(images=PIL_Image, return_tensors="pt")
  outputs = deit_model(**inputs)
  logits = outputs.logits

  top_5_class_idx = np.array(logits.argsort())[0][::-1][:5]

  top_5_class = [( vit_model.config.id2label[str(class_idx)],
                  norm( logits[0][class_idx].item()/10 ) ) for class_idx in top_5_class_idx]

  return top_5_class

In [ ]:
def ensemble_2(t1, t2):
  if t1[0] == t2[0]:
    return ( t1[0], (t1[1] + t2[1])/2 )
  else:
    if t1[1] >= t2[1]:
      return ( t1[0], t1[1] )
    if t1[1] < t2[1]:
      return ( t2[0], t2[1] )

In [ ]:
def predict(PIL_Image):
  deit_predicition = run_deit(PIL_Image)
  vit_prediction = run_vit(PIL_Image)

  ensemble_prediction = []

  for idx in range(len(vit_prediction)):
    ensemble_prediction.append(ensemble_2(vit_prediction[idx], deit_predicition[idx]))

  return ensemble_prediction

Statistics:

In [ ]:
ex = ds["train"][3123]
ex_image = ex['pixel_values']
ex_label = (ds['test'].features['label']).int2str(ex['label'])

print(ensemble_2(ex_image))
print(ex_label)

('basal cell carcinoma', 0.9919523239135742)
basal cell carcinoma


In [ ]:
sum = {label: 0 for label in ds["test"].features["label"].names}
total = {label: [] for label in ds["test"].features["label"].names}


for test_ex_index in tqdm(range(len(test_ds))):
  test_ex = test_ds[test_ex_index]

  test_ex_image = test_ex["pixel_values"]
  test_ex_label = (ds['test'].features['label']).int2str(test_ex['label'])

  prediction = ensemble_2(test_ex_image)

  total[test_ex_label] += 1

  if prediction[0] == test_ex_label:
    sum[test_ex_label] += 1

100%|██████████| 6051/6051 [22:15<00:00,  4.53it/s]


In [ ]:
accuracy = {}

for label in ds["test"].features["label"].names:
  accuracy[label] = sum[label]/total[label]

In [ ]:
num_images = {label: 0 for label in ds["test"].features["label"].names}

for ex in test_ds:
  num_images[ (ds['test'].features['label']).int2str(ex['label']) ] += 1

In [ ]:
weighted_accuracy = 0

for disease, acc in accuracy.items():
  weighted_accuracy += acc * (num_images[disease]/len(test_ds))

In [ ]:
print(weighted_accuracy)

0.7909436456784006


In [ ]:
import json
with open("accuracy.json", "w") as f:
  json.dump(accuracy, f)

In [ ]:
with open("num_test_images.json", "w") as f:
  json.dump(num_images, f)

In [ ]:
num_images["Normal Skin"]

76

In [ ]:
total_disease = 0
predict_disease = 0

for test_ex_index in tqdm(range(len(test_ds))):
  test_ex = test_ds[test_ex_index]

  test_ex_image = test_ex["pixel_values"]
  test_ex_label = (ds['test'].features['label']).int2str(test_ex['label'])

  if test_ex_label != "Normal Skin":
    prediction = ensemble_2(test_ex_image)

    total_disease += 1

    if prediction[0] == test_ex_label:
      predict_disease += 1

100%|██████████| 6051/6051 [21:53<00:00,  4.61it/s]


In [ ]:
print(f"Sensativity: {100 * (predict_disease/total_disease)}")

Sensativity: 78.82845188284519


In [ ]:
print(total_disease)
print(predict_disease)

5975
4710
